In [1]:
import pandas as pd
import numpy as np
import cobra
import re

Load data:

In [6]:
data_dir = '//imsbnas.ethz.ch/Sauer1/users/Xavier/3_Karl_data/'
metabolites = pd.read_excel(data_dir+"merged_metabolites.xlsx",index_col="name")
reactions = open(data_dir+'reactions_GS2.txt').read().splitlines()
mapping = pd.read_table(data_dir+'bigg_models_reactions.txt',index_col='bigg_id')
regs = pd.read_excel(data_dir+'SMRN_allorgs.xlsx',index_col='EC_number')

Calculate the metabolite variance (metabolites with higher variances are suposed to endorse more allosteric interactions).

In [7]:
variances = np.nanvar(metabolites,1)

array([  1.30708404e-02,   7.57103840e-02,   1.76014199e-02,
         5.27847491e-03,   4.32960140e-02,   3.40669263e-02,
         1.49269820e-02,   4.41080342e-02,   6.27579117e-02,
         4.11540055e-01,   4.67012567e-02,   2.66029833e-01,
         1.56188382e-01,   1.88250477e+01,   4.62021003e-01,
         8.66253139e-03,   1.45623544e-02,   6.80306645e-03,
         3.86415672e-02,   1.21500183e-01,   8.27177202e-03,
         1.05900398e-02,   1.94850028e-02,   2.75812167e-02,
         1.43457558e-02,   3.63606972e-01,   8.41759234e-02,
         1.55157817e-01,   5.77073378e+00,   1.51713907e-02,
         5.64540204e-03,   1.02694187e-01,   9.10635313e-02,
         4.74859157e-03,   1.97186805e-01,   4.28889592e-02,
         7.63096422e-02,   9.60373722e-02,   1.73359661e-02,
         2.10336493e-02,   6.45810538e-01,   1.51012188e-02,
         6.47480750e+00,   1.11751909e-02,   5.25694637e-03,
         1.25335013e-02,   4.65451278e-02,   1.79655619e-01,
         1.44580171e-01,

For each reaction, look at metabolites that have been previously reported in any organism. Incorporate this prior data in a logistic regression to calculate the probability of each metabolite to be a real regulator.

In [17]:
threshold = 100 #100 candidates out of 280*2
shortlist = pd.DataFrame(columns=['metab','mode','scores','rxn_id'])
for rxn in reactions:
    rxn_df = pd.DataFrame({'metab':list(metabolites.index)+list(metabolites.index),'mode':['+']*metabolites.shape[0]+['-']*metabolites.shape[0],\
              'scores':list(variances)+list(variances),'rxn_id':[rxn.lower()]*metabolites.shape[0]*2})
    ec = re.findall('[1-6]+.[0-9]+.[0-9]+.[0-9]+',mapping.loc[rxn,'database_links'])
    rxn_regs = pd.DataFrame(columns=regs.columns)
    for i in ec:
        if i in regs.index:
            rxn_regs = rxn_regs.append(regs.loc[ec],ignore_index=True)
    n_allorgs = [np.sum(np.logical_and([rxn_regs['bigg.metabolite']==rxn_df.loc[s,'metab']],[rxn_regs['Mode']==rxn_df.loc[s,'mode']])) for s in list(rxn_df.index.values)]
    n_allorgs = np.array(n_allorgs)*2.0+1.0
    rxn_df['scores'] = rxn_df['scores']*n_allorgs
    rxn_df = rxn_df.sort_values(by='scores',ascending=False)
    shortlist = shortlist.append(rxn_df.iloc[0:threshold,:],ignore_index=True)
shortlist

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9 FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,metab,mode,rxn_id,scores
0,akg,-,a5piso,24.024672
1,akg,+,a5piso,24.024672
2,2obut,+,a5piso,18.825048
3,2obut,-,a5piso,18.825048
4,mev__R,+,a5piso,15.135206
5,mev__R,-,a5piso,15.135206
6,5aizc,-,a5piso,6.474808
7,5aizc,+,a5piso,6.474808
8,3php,+,a5piso,5.770734
9,3php,-,a5piso,5.770734


Save shorlist

In [18]:
shortlist.to_csv('//imsbnas.ethz.ch/Sauer1/users/Xavier/11_Karl_pairwise/shortlist.csv')